In [1]:
!pip install gurobipy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 76.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from gurobipy import Model, GRB
import time
import random

In [3]:
!kaggle datasets download -d mexwell/telecom-shanghai-dataset

Dataset URL: https://www.kaggle.com/datasets/mexwell/telecom-shanghai-dataset
License(s): other
 92%|█████████████████████████████████████▊   | 202M/219M [00:02<00:00, 141MB/s]
100%|█████████████████████████████████████████| 219M/219M [00:02<00:00, 102MB/s]


In [4]:
!unzip /kaggle/working/telecom-shanghai-dataset.zip

Archive:  /kaggle/working/telecom-shanghai-dataset.zip
  inflating: data_10.110.15.xlsx     
  inflating: data_10.1610.31.xlsx    
  inflating: data_11.111.15.xlsx     
  inflating: data_11.1611.30.xlsx    
  inflating: data_6.16.15.xlsx       
  inflating: data_6.166.30.xlsx      
  inflating: data_7.167.31.xlsx      
  inflating: data_7.17.15.xlsx       
  inflating: data_8.168.31.xlsx      
  inflating: data_8.18.15.xlsx       
  inflating: data_9.169.30.xlsx      
  inflating: data_9.19.15.xlsx       


In [5]:
dir_path = "/kaggle/working/"
df = pd.DataFrame()
for file_name in os.listdir(dir_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(dir_path,file_name)
        
        temp_df = pd.read_excel(file_path,engine='openpyxl')
        df = pd.concat([df,temp_df],ignore_index = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6952921 entries, 0 to 6952920
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   month       int64         
 1   date        int64         
 2   start time  datetime64[ns]
 3   end time    datetime64[ns]
 4   latitude    float64       
 5   longitude   float64       
 6   user id     object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(1)
memory usage: 371.3+ MB


In [6]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6952921 entries, 0 to 6952920
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   month       6952921 non-null  int64         
 1   date        6952921 non-null  int64         
 2   start time  6952921 non-null  datetime64[ns]
 3   end time    6952921 non-null  datetime64[ns]
 4   latitude    6236620 non-null  float64       
 5   longitude   6236620 non-null  float64       
 6   user id     6952921 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(1)
memory usage: 371.3+ MB


In [7]:
df_clean = df.dropna(subset = ['latitude','longitude'])
df_clean['latitude_norm'] = (df['latitude'] - df['latitude'].min()) / (df['latitude'].max() - df['latitude'].min())
df_clean['longitude_norm'] = (df['longitude'] - df['longitude'].min()) / (df['longitude'].max() - df['longitude'].min())
df.info(show_counts=True)
df.head()


/tmp/ipykernel_24/4133840982.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['latitude_norm'] = (df['latitude'] - df['latitude'].min()) / (df['latitude'].max() - df['latitude'].min())
/tmp/ipykernel_24/4133840982.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['longitude_norm'] = (df['longitude'] - df['longitude'].min()) / (df['longitude'].max() - df['longitude'].min())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6952921 entries, 0 to 6952920
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   month       6952921 non-null  int64         
 1   date        6952921 non-null  int64         
 2   start time  6952921 non-null  datetime64[ns]
 3   end time    6952921 non-null  datetime64[ns]
 4   latitude    6236620 non-null  float64       
 5   longitude   6236620 non-null  float64       
 6   user id     6952921 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(1)
memory usage: 371.3+ MB


,month,date,start time,end time,latitude,longitude,user id
0,201411,1,2014-11-01 09:18:08,2014-11-01 09:34:25,NaN,NaN,57ec951bde737466587feaed0d806a11
1,201411,1,2014-11-01 18:12:01,2014-11-01 18:22:34,NaN,NaN,cd6e779892f6c03e22211b25409ba75e
2,201411,1,2014-11-01 18:12:02,2014-11-01 18:21:00,NaN,NaN,cd6e779892f6c03e22211b25409ba75e
3,201411,1,2014-11-01 15:33:03,2014-11-01 15:34:43,NaN,NaN,b341b594239f8b8d36ed78394426af68
4,201411,1,2014-11-01 15:19:10,2014-11-01 15:20:10,NaN,NaN,b341b594239f8b8d36ed78394426af68
